In [1]:
import torch
from sklearn.datasets import make_blobs
from algorithms import fit_by_sgd

In [2]:
class SimpleModel(torch.nn.Module):
    """
    Simple classification model. 1 hidden layer with ReLU activation
    """

    def __init__(self, input_size: int = 2, hidden_size: int = 100, n_classes: int = 3, print_function=print):
        super().__init__()
        self.w1 = torch.nn.Linear(input_size, hidden_size)
        self.w2 = torch.nn.Linear(hidden_size, n_classes)
        self.softmax = torch.nn.Softmax(dim=1)
        self.print = print_function

    def __call__(self, x):
        return self.softmax(torch.selu(self.w2(self.w1(x))))

    def loss(self, x, y):
        cross_entropy_loss = torch.nn.CrossEntropyLoss()
        return cross_entropy_loss(self(x), y)

In [18]:
X, Y = make_blobs(n_samples=10_000, centers=2, cluster_std=5)
X, Y = torch.FloatTensor(X), torch.tensor(Y, dtype=torch.long)

model = SimpleModel(n_classes=2)

In [19]:
model, history = fit_by_sgd(model, X, Y, epochs=10000, batch_size=1)

epoch:     1 | Q:  1.0499
epoch:     2 | Q:  0.9274
epoch:     7 | Q:  1.0258
epoch:    21 | Q:  0.7475
epoch:    59 | Q:  0.8164
epoch:   166 | Q:  1.1032
epoch:   464 | Q:  0.4538
epoch:  1291 | Q:  0.4208
epoch:  3593 | Q:  0.4691
epoch:  10000 | Q:  0.5101


In [20]:
import plotly.express as px

px.line(y=history['q_loss'])

In [21]:
from Classification.metrics import auc_roc
from Classification.metrics import roc_curve_plot

auc_roc(Y, model(X).detach()[:, 1], n_thresholds=1000)

0.8850728273391724

In [22]:
roc_curve_plot(Y, model(X)[:, 1].detach())